In [1]:
import os
import numpy as np
import pandas as pd
from pydub import AudioSegment
from tqdm.autonotebook import tqdm
import IPython.display as ipd
import librosa
import librosa.display
import glob, scipy, random
import noisereduce as nr
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import random

probability = 0.3

def pre(path):
    longest = 0
    l = ''
    for _, n in enumerate(['/1','/2','/3','/4','/5']):
        for audio in [(path+n + "/" + fname) for fname in os.listdir(path+n)]:
            data, sampling_rate = librosa.load(audio, sr = 16000)
            if not os.path.exists(path+'/1-wav'):
                hpss = librosa.effects.hpss(data)
                data, ind = librosa.effects.trim(hpss[1], top_db=40)
            if len(data) > longest:
                l = audio
            longest = max(longest, len(data))
            librosa.output.write_wav(audio, data, sampling_rate)
    print(l)
    return longest

def configureData(path, longest):
    for _, n in enumerate(['/1','/2','/3','/4','/5']):
        if not os.path.exists(path+n+'-wav'):
            os.mkdir(path + n + '-wav')
            convertToWav([(path+n + "/" + fname) for fname in os.listdir(path+n)], longest)

def convertToWav(audio_paths, longest):
    for audios in audio_paths:
        sound = AudioSegment.from_file(audios)
        reverseindex = len(audios)-1 - audios[::-1].index("/")
        audios = audios[:reverseindex] + '-wav' + audios[reverseindex:-3] + 'wav'
        sound += (-20 - sound.dBFS)
        sound.export(audios, format="wav")
        data, sampling_rate = librosa.load(audios, sr = 16000)

        data = list(data)
        data = [0.0] * int((longest - len(data))/2) + data + [0.0] * (longest - len(data) - int((longest - len(data))/2))
        data = np.array(data)
        librosa.output.write_wav(audios, data, sampling_rate)
        for i, shft in enumerate([0.05, 0.1, 0.15, 0.25, 0.35]):
            if random.random() < probability: continue
            shiftLeft = shifting(data, sampling_rate, shft, 'left')
            shiftRight = shifting(data, sampling_rate, shft, 'right')
            librosa.output.write_wav(audios[:-4] + str(i) +'1' + audios[-4:], shiftLeft, sampling_rate)
            librosa.output.write_wav(audios[:-4] + str(i) +'2' + audios[-4:], shiftRight, sampling_rate)
            # Pitch
            pitch = librosa.effects.pitch_shift(data, sampling_rate, shft)
            librosa.output.write_wav(audios[:-4] + str(i) +'3' + audios[-4:], pitch, sampling_rate)
    
            noise = np.random.randn(longest)
            noiseDat = data + noise * 0.05
            noiseDat = noiseDat.astype(type(data[0]))
            librosa.output.write_wav(audios[:-4] + str(i) +'4' + audios[-4:], noiseDat, sampling_rate)
        
        

def shifting(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    augmented_data = np.roll(data, shift)
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

def loadData(path):
    paths = {}
    for i, n in enumerate(['/1-wav','/2-wav','/3-wav','/4-wav','/5-wav']):
        paths[i+1] = [(path+n + "/" + fname) for fname in os.listdir(path+n)]
    return paths


def splitData(prc):
    test_wavs = {}
    for key in wavs.keys():
        test_wavs[key] = []
        size = int(prc*len(wavs[key]))
        random.shuffle(wavs[key])
        for i in range(size):
            test_wavs[key].append(wavs[key].pop())
    return test_wavs

longest = pre(os.getcwd().replace("\\", "/"))
configureData(os.getcwd().replace("\\", "/"), longest)
wavs = loadData(os.getcwd().replace("\\", "/"))
test_wavs = splitData(0.2)
len(wavs[1])

C:\Users\alexp\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


C:/Users/alexp/Desktop/NumbersRecognition/4/zviad-noza-4d.m4a


1692

In [2]:
data, sampling_rate = librosa.load(wavs[1][0], sr = 16000)
ipd.Audio(data, rate=sampling_rate)

In [3]:
leng = len(librosa.feature.mfcc(data, sr=16000).flatten())

In [4]:
def getYector(num):
    v = np.zeros((1,5))
    v[:,num-1] = 1
    return v

def configureX(data):
    size = len(data[1])+len(data[2])+len(data[3])+len(data[4])+len(data[5]) 
    ones = np.ones((size, 1))
    i, x = 0, np.zeros((size, leng))
    for n in data.keys():
        for path in data[n]:
            x[i] = np.matrix(librosa.feature.mfcc(librosa.load(path, sr = 16000)[0], sr=16000).flatten())
            i += 1
    x = x.astype(np.float64)
    return x 

def configureY(data):
    size = len(data[1])+len(data[2])+len(data[3])+len(data[4])+len(data[5]) 
    i, y = 0, np.zeros((size,5))
    for n in data.keys():
        for path in data[n]:
            y[i] = getYector(n)
            i += 1
    return y

X = configureX(wavs)
y = configureY(wavs)
X.shape, y.shape

((8178, 1900), (8178, 5))

In [5]:
class NeuralNetwork:
    
    def __init__(self, layers):
        self.layers = layers
        self.weights = [None] * len(self.layers)
        self.bias = [None] * len(self.layers)
        self.n = float(X.shape[0])
        for i in range(1, len(self.layers)):
            self.bias[i] = np.zeros((self.layers[i], 1))
            self.weights[i] = np.random.randn(self.layers[i], self.layers[i - 1]) * np.sqrt( 2 /
               (self.layers[i] + self.layers[i - 1]))
            
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, z):
        sig = self.sigmoid(z)
        return sig * (1 - sig)
    
     
    def relu(self, z):
        return np.maximum(0.01*z, z)
    
    def relu_derivative(self, z):
        dx = np.ones_like(z)
        dx[z < 0] = 0.01
        return dx
 
    def softmax(self, z):
        exp = np.exp(z - np.max(z))
        return exp / exp.sum(axis=0, keepdims=True)

    def forward(self, X):
        memorize = {}
        a = X.T
        for i in range(1, len(self.layers)):
            z = self.weights[i] @ a + self.bias[i]
            if i == len(self.layers) - 1:
                a = self.softmax(z)
            else:
                a = self.sigmoid(z)
            memorize["z" + str(i)] = z
            memorize["a" + str(i)] = a
        return a, memorize
 
    def backpropagation(self, X, y, learning_rate, memorize):
        length = len(self.layers) - 1
        a = memorize["a" + str(length)]
        error = y.T - a 
        deltaA = self.weights[length].T @ error
        deltaW = error @ memorize["a" + str(length - 1)].T / self.n
        deltaB = np.sum(error, axis=1, keepdims=True) / self.n
        
        self.weights[length] += learning_rate * deltaW + (self.weights[length] * learning_rate) / self.n
        self.bias[length] += learning_rate * deltaB
 
        memorize["a0"] = X.T
        for l in reversed(range(1, length)):
            error = deltaA * self.sigmoid_derivative(memorize["z" + str(l)])
            deltaA = self.weights[l].T @ error
            deltaW = error @ memorize["a" + str(l - 1)].T / self.n
            deltaB = np.sum(error, axis=1, keepdims=True) / self.n
            
            self.weights[l] += learning_rate * deltaW + (self.weights[l] * learning_rate) / self.n
            self.bias[l] += learning_rate * deltaB

 
    def train(self, X, y, learning_rate, iterations):
        for i in range(iterations):
            _, store = self.forward(X)
            self.backpropagation(X, y, learning_rate, store)
            if i % 100 == 0:
                print("Train Accuracy on " + str(i) + "th iteration: ", self.predict(X, y))
 
 
    def predict(self, X, y):
        pred, _ = self.forward(X)
        predicted = np.argmax(pred, axis=0)
        real = np.argmax(y, axis=1)
        return float((predicted == real).mean()) * 100 

In [6]:
layers = [X.shape[1], 500, 200, 5]
X = np.array(X)
y = np.array(y)
nn = NeuralNetwork(layers)
nn.train(X, y, learning_rate=0.01, iterations=3312)
print("Final Train Accuracy:", nn.predict(X, y))
X = np.array(configureX(test_wavs))
y = np.array(configureY(test_wavs))
print("Test Accuracy:", nn.predict(X, y))

C:\Users\alexp\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


Train Accuracy on 0th iteration:  20.371729029102468
Train Accuracy on 100th iteration:  47.652237710931765
Train Accuracy on 200th iteration:  50.94155050134507
Train Accuracy on 300th iteration:  52.384446074834926
Train Accuracy on 400th iteration:  52.50672536072389
Train Accuracy on 500th iteration:  55.53925165077036
Train Accuracy on 600th iteration:  57.60577158229396
Train Accuracy on 700th iteration:  59.5255563707508
Train Accuracy on 800th iteration:  61.164098801663
Train Accuracy on 900th iteration:  64.33113230618733
Train Accuracy on 1000th iteration:  65.2971386647102
Train Accuracy on 1100th iteration:  65.90853509415506
Train Accuracy on 1200th iteration:  67.339202739056
Train Accuracy on 1300th iteration:  65.99413059427734
Train Accuracy on 1400th iteration:  67.98728295426754
Train Accuracy on 1500th iteration:  68.76987038395696
Train Accuracy on 1600th iteration:  70.38395695769137
Train Accuracy on 1700th iteration:  73.18415260454879
Train Accuracy on 1800th 

In [7]:
with open('model.sav', 'wb') as model:
    pickle.dump(layers, model)
    pickle.dump(nn.weights, model)
    pickle.dump(nn.bias, model)
    pickle.dump(longest, model)